In [1]:
from google.colab import drive
drive.mount('/content/drive')


%cd /content/drive/MyDrive/ST-GAT

Mounted at /content/drive
/content/drive/MyDrive/ST-GAT


In [2]:
import pandas as pd
import datetime


n_his = 24
n_pred = 12

col = pd.read_csv('./dataset/col_ap.csv', header=None).values.flatten()

specific_date = datetime.datetime(2021, 5, 11)

idx = pd.date_range(start=(specific_date), end=(specific_date + datetime.timedelta(days=1)), freq='5min').values[(n_his + n_pred -1):-1]
idx = pd.DatetimeIndex(data=idx, tz='Etc/GMT+2')

week_num = datetime.date.today().weekday() # week_num = 0 is Monday, and 6 is Sunday.
seasonal = pd.read_csv('./dataset/seasonal_week.csv', header=None).values[(week_num * 288): ((week_num +1)* 288)][(n_his + n_pred -1):]
df_seasonal = pd.DataFrame(seasonal, index=idx, columns=col)

csv_file_path = './save/seasonal.csv'
df_seasonal.to_csv(csv_file_path, index=False)



In [3]:
!pip install torch_geometric

import torch_geometric
from torch_geometric.nn.conv import GATConv

from load_data import *
import pandas as pd
import torch
from stgat import *
from utils import *
import numpy as np
import datetime
from sklearn.preprocessing import StandardScaler

matrix_path = "dataset/03062021_M_96.csv"

n_his = 24
n_pred = 12
n_route = 96
Ks, Kt = 3, 3
blocks = [[1, 32, 64], [64, 32, 128]]
drop_prob = 0

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

n_his = 24
n_pred = 12
day_slot = 288

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.3 MB/s eta 0:00:00


In [4]:


def calculate_edge_index_from_adj_matrix(csv_file):
  """
  Calculates the edge index from an adjacency matrix in a CSV file.

  Args:
      csv_file (str): Path to the CSV file containing the adjacency matrix.

  Returns:
      numpy.ndarray: A 2D NumPy array of shape (num_edges, 2) representing the edge index.
                        - Each row contains [source_node_index, target_node_index] for an edge.
  """
  # Read the adjacency matrix
  adj_matrix = pd.read_csv(csv_file, header=None).values

  # Find non-zero elements (assuming they represent edges)
  edges = adj_matrix.nonzero()

  # Combine source and target node indices into a single 2D array
  edge_index = np.vstack([edges[0], edges[1]])

  edge_index_tensor = torch.LongTensor(edge_index)

  return edge_index_tensor

# Example usage
# csv_file = "./dataset/03062021_M_96.csv"
edge_index = calculate_edge_index_from_adj_matrix(matrix_path)

edge_index = edge_index.to(device)

print("Edge Index:", edge_index.shape)


Edge Index: torch.Size([2, 8624])


In [5]:
data = pd.read_csv("./dataset/03062021_Pred_adj_96.csv", header=None).values.astype(float)

mean_values = data.mean()
std_values = data.std()

scaler = StandardScaler()

data = scaler.fit_transform(data)

x_ , y_  = data_transform(data, n_his, n_pred, day_slot, device)

print(device)

# model_state_dict = torch.load('./save/model.pt')
# model_state_dict = torch.load('./save/model_simple_gn.pt')
# model_state_dict = torch.load('./save/simple_gn_HPO.pt')
model_state_dict = torch.load('./save/GAT_model_2.pt')


num_layers = 2
in_features , hidden_channels = n_route , n_route
# hidden_channels = 64
out_features = 96


model = BasicGAT(in_features, hidden_channels, num_layers, out_features)  # Assuming your STGCN model class is named STGCN
model = model.to(device)

# print(model.device)

model.load_state_dict(model_state_dict)

# Step 4: Perform inference
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    predicted_outputs = model(x_ , edge_index)

reshaped_pred = predicted_outputs.reshape(253, 96)
numpy_array = reshaped_pred.cpu().numpy()
# numpy_array = reshaped_pred.numpy()

numpy_array = numpy_array * std_values + mean_values

# csv_file_path = './save/output.csv'
# np.savetxt(csv_file_path, numpy_array, delimiter=',')

# print(predicted_outputs.size())
print(numpy_array.shape)
# print(y_)

# col_ap = pd.read_csv("./dataset/col_ap.csv", header=None).values.flatten()

# Step 2: Create timestamps
num_timesteps = numpy_array.shape[0]  # Assuming the number of timesteps is the first dimension of numpy_array
time_diff = datetime.timedelta(minutes=5)  # Assuming each timestep is 5 minutes
# specific_date = datetime.datetime(2021, 5, 11)  # Specific date for the first timestamp
specific_date = datetime.datetime(2021, 5, 11, 2, 55, tzinfo=datetime.timezone(datetime.timedelta(hours=-2)))

timestamps = [specific_date + i * time_diff for i in range(num_timesteps)]

# Step 3: Combine timestamps and predicted data into a DataFrame
data = np.concatenate((np.array(timestamps).reshape(-1, 1), numpy_array), axis=1)

# Create a DataFrame
# df_adj = pd.DataFrame(data, columns=['timestamp'] + col_ap.tolist())
df_adj = pd.DataFrame(numpy_array, index=idx, columns=col)

col = pd.read_csv('./dataset/col_ap.csv', header=None).values.flatten()

specific_date = datetime.datetime(2021, 5, 11)

idx = pd.date_range(start=(specific_date), end=(specific_date + datetime.timedelta(days=1)), freq='5min').values[(n_his + n_pred -1):-1]
idx = pd.DatetimeIndex(data=idx, tz='Etc/GMT+2')

week_num = datetime.date.today().weekday() # week_num = 0 is Monday, and 6 is Sunday.
seasonal = pd.read_csv('./dataset/seasonal_week.csv', header=None).values[(week_num * 288): ((week_num +1)* 288)][(n_his + n_pred -1):]
df_seasonal = pd.DataFrame(seasonal, index=idx, columns=col)
# df_adj = pd.DataFrame(data, index=idx, columns=col)

df = df_adj + df_seasonal

# df["Timesteps"]  = timestamps
df.insert(0, 'Timesteps', timestamps)


# Save the DataFrame to a CSV file
csv_file_path = './save/predictions_gat3.csv'
df.to_csv(csv_file_path, index=False)

print(f"CSV file saved at '{csv_file_path}'")

cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


(253, 96)
CSV file saved at './save/predictions_gat3.csv'
